---
# Section 5.3: The Power Method and Some Simple Extensions
---

Let $A \in \mathbb{C}^{n \times n}$ be a matrix with _linearly independent_ eigenvectors

$$
v_1, \ldots, v_n
$$

and corresponding eigenvalues

$$
\lambda_1, \ldots, \lambda_n
$$

(i.e., $A v_i = \lambda_i v_i$, for $i=1,\ldots,n$) ordered such that

$$
|\lambda_1| \ge |\lambda_2| \ge \cdots \ge |\lambda_n|.
$$

We say that $A$ has a **dominant eigenvalue** if
 
$$
|\lambda_1| > |\lambda_2|.
$$

---

## The Power Method

The basic idea of the **power method** is to pick a vector $q \in \mathbb{C}^n$ and compute the sequence

$$
q,\ A q,\ A^2 q,\ A^3 q,\ \ldots.
$$

Since the eigenvectors $v_1,\ldots,v_n$ form a basis for $\mathbb{C}^n$, we have that

$$
q = c_1 v_1 + \cdots + c_n v_n.
$$

For a random $q$, we expect $c_1 \ne 0$.

Then

$$
\begin{align}
A q
&= c_1 A v_1 + \cdots + c_n A v_n \\
&= c_1 \lambda_1 v_1 + \cdots + c_n \lambda_n v_n
\end{align}
$$

and

$$
\begin{align}
A^2 q
&= c_1 \lambda_1 A v_1 + \cdots + c_n \lambda_n A v_n \\
&= c_1 \lambda_1^2 v_1 + \cdots + c_n \lambda_n^2 v_n.
\end{align}
$$

In general, we have

$$
A^j q = c_1 \lambda_1^j v_1 + \cdots + c_n \lambda_n^j v_n
$$

and

$$
\frac{A^j q}{\lambda_1^j} = c_1 v_1 + c_2 \left(\frac{\lambda_2}{\lambda_1}\right)^j v_2 + \cdots + c_n \left(\frac{\lambda_n}{\lambda_1}\right)^j v_n.
$$

Letting

$$
q_j = \frac{A^j q}{\lambda_1^j},
$$

we have

$$
\begin{align}
\| q_j - c_1 v_1 \|
&= \left\| c_2 \left(\frac{\lambda_2}{\lambda_1}\right)^j v_2 + \cdots + c_n \left(\frac{\lambda_n}{\lambda_1}\right)^j v_n \right\| \\
&\le |c_2| \left|\frac{\lambda_2}{\lambda_1}\right|^j \|v_2\| + \cdots + |c_n| \left|\frac{\lambda_n}{\lambda_1}\right|^j \|v_n\| \\
&\le \left|\frac{\lambda_2}{\lambda_1}\right|^j \big(|c_2| \|v_2\| + \cdots + |c_n| \|v_n\|\big).
\end{align}
$$

Now suppose $|\lambda_1| > |\lambda_2|$. Then

$$
\left|\frac{\lambda_2}{\lambda_1}\right| < 1.
$$

Therefore,

$$
\left|\frac{\lambda_2}{\lambda_1}\right|^j \to 0 \quad \text{as} \ j \to \infty.
$$

Thus, $\| q_j - c_1 v_1 \| \to 0$ as $j \to \infty$, so we conclude that

$$
q_j \to c_1 v_1 \quad \text{as $j \to \infty$.}
$$

---

## Example

A matrix $A \in \mathbb{R}^{n \times n}$ with nonnegative entries whose columns sum to one is called a [**stochastic matrix**](https://en.wikipedia.org/wiki/Stochastic_matrix). Such a matrix often has a dominant eigenvalue of $\lambda_1 = 1$ and the corresponding eigenvector $v_1$ represents a _stationary probability vector_.

Use the power method to compute the eigenvector $v_1$.

In [199]:
using LinearAlgebra

n = 4
A = rand(n,n)

# Scale A so each column sums to one
A ./= sum(A,dims=1)

4×4 Matrix{Float64}:
 0.217455  0.42322   0.352961  0.233545
 0.168122  0.188025  0.126703  0.335685
 0.231962  0.115256  0.337741  0.187925
 0.38246   0.273499  0.182594  0.242845

In [200]:
sum(A,dims=1)

1×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0

In [271]:
function power_method(A; tol=eps(), maxiter=100)
    m, n = size(A)
    n == m || error("Matrix must be square.")
    
    q = randn(n)
    qold = similar(q)

    k = 0
    done = false
    while !done && k <= maxiter
        k += 1
        copy!(qold, q)
        mul!(q, A, qold)  # q = A*qold
        @show k, norm(q - qold)
        done = norm(q - qold) < tol
        normalize!(q)
    end

    return q
end

power_method (generic function with 1 method)

In [280]:
q = power_method(A)

(k, norm(q - qold)) = (1, 1.014377320377164)
(k, norm(q - qold)) = (2, 0.308499101992868)
(k, norm(q - qold)) = (3, 0.04975802334977585)
(k, norm(q - qold)) = (4, 0.009776682575648595)
(k, norm(q - qold)) = (5, 0.0017397445513524948)
(k, norm(q - qold)) = (6, 0.00026531539949659554)
(k, norm(q - qold)) = (7, 4.819203211024785e-5)
(k, norm(q - qold)) = (8, 9.12853736410714e-6)
(k, norm(q - qold)) = (9, 1.4579830783242e-6)
(k, norm(q - qold)) = (10, 2.41541905640608e-7)
(k, norm(q - qold)) = (11, 4.623937856379362e-8)
(k, norm(q - qold)) = (12, 8.05513292102129e-9)
(k, norm(q - qold)) = (13, 1.2661738868307013e-9)
(k, norm(q - qold)) = (14, 2.2893869803408798e-10)
(k, norm(q - qold)) = (15, 4.378917512769234e-11)
(k, norm(q - qold)) = (16, 6.955340106752214e-12)
(k, norm(q - qold)) = (17, 1.1450855776469353e-12)
(k, norm(q - qold)) = (18, 2.3388175473915523e-13)
(k, norm(q - qold)) = (19, 3.915535627258809e-14)
(k, norm(q - qold)) = (20, 6.1074881222063774e-15)
(k, norm(q - qold)) = (21,

4-element Vector{Float64}:
 0.5830987013178677
 0.41499146942146264
 0.43214785516322773
 0.5486585605881436

In [267]:
A*q - q

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 1.1102230246251565e-16

In [268]:
eigen(A)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
4-element Vector{ComplexF64}:
 -0.10452144834580698 - 0.13799339417016865im
 -0.10452144834580698 + 0.13799339417016865im
   0.1951094977020419 + 0.0im
   0.9999999999999991 + 0.0im
vectors:
4×4 Matrix{ComplexF64}:
  0.646636-0.0im        …  -0.0100158+0.0im  -0.583099+0.0im
 -0.187001-0.451364im         0.49673+0.0im  -0.414991+0.0im
 -0.181834-0.0303076im      -0.806701+0.0im  -0.432148+0.0im
 -0.277802+0.481671im        0.319987+0.0im  -0.548659+0.0im